<a href="https://colab.research.google.com/github/michelle-shih/2022CALISE/blob/main/111_CALISE_Project_Test_CDA111004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 111年 CALISE 中華圖書資訊學教育學會大數據競賽

## 題目：書目資料主題詞預測

## 測試專案格式
本測試專案將做為評審時**實際執行測試**之格式，請記得將所有您所需要的套件、環境與模型進行設定。

# 1.測試(Testing)
您可於此處使用測試資料集並載入訓練好的模型進行測試。
此處將作為評審時實際執行之部分，所有需要導入的套件、函式庫、資料處理與模型載入等等亦需包含於此部分。
請印出前10筆分析結果，以利評審判斷。
* 模型或其他您設計需要預先下載的東西請記得置放於可以下載的空間中(如Github...等)。

In [ ]:
import os
#讀檔案
#掛雲端
from google.colab import drive
drive.mount('/content/drive')

##出現提示欄進行授權

os.chdir('/content/drive/My Drive/111_CALISE_CDA111004') #切換該目錄
#https://drive.google.com/drive/folders/1zoSxsPrDdX_7yepTgpBBELRfRGNV0f_5?usp=sharing
os.listdir() #確認目錄內容

Mounted at /content/drive


['NotoSansCJKtc-Black.otf',
 'NotoSansCJKtc-Light.otf',
 'NotoSansCJKtc-DemiLight.otf',
 'NotoSansCJKtc-Bold.otf',
 'NotoSansCJKtc-Thin.otf',
 'NotoSansCJKtc-Medium.otf',
 'NotoSansMonoCJKtc-Bold.otf',
 'NotoSansMonoCJKtc-Regular.otf',
 'README',
 'LICENSE_OFL.txt',
 'NotoSansCJKtc-hinted.zip',
 'NotoSansCJKtc-hinted.zip.1',
 'NotoSansCJKtc-hinted.zip.2',
 'data',
 'variables',
 'assets',
 'ckiptagger_model',
 'keras_metadata.pb',
 'saved_model.pb',
 'our_model.h5',
 'pus_model.h5',
 '111_CALISE_Project_Test_CDA111004.ipynb',
 '111_CALISE_Project_Train_CDA111004.ipynb',
 'test_class_num.npy',
 'test_subject_CBOW.wv',
 'test_subjects.npy',
 'test_y_train_numpy_array.npy',
 'test_language_code.npy',
 'test_title_CBOW.wv',
 'test_title.npy',
 'nice_model.h5',
 'class_num.npy',
 'subject_CBOW.wv',
 'subjects.npy',
 'y_train_numpy_array.npy',
 'language_code.npy',
 'title_CBOW.wv',
 'title.npy']

#### test資料前處理

In [ ]:
#套件載入
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import regex as re # 處理文字比對、替換等工作的套件
from IPython import display
import matplotlib.dates as mdates
from ipywidgets import Text, IntText
from datetime import datetime # 處理日期時間資料的套件
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from numpy import array
from keras.preprocessing.sequence import pad_sequences
from gensim.models import word2vec
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
# 安裝中研院資訊所詞知識庫小組(ckip)的斷詞系統
!pip install -U ckiptagger[tf,gdown]
# 載入ckiptagger套件中所需程式
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# 清除執行結果
display.clear_output()

In [ ]:
#主題詞dictionary
subdict = {0:'United States',1:'History',2:'中國',3:'Politics and government',4:'Great Britain',5:'Congresses',
           6:'Philosophy',7:'China',8:'傳記',9:'History and criticism',10:'歷史',11:'Social aspects',
           12:'Biography',13:'Foreign relations',14:'臺灣',15:'Europe',16:'Economic conditions',
           17:'Psychology',18:'論文',19:'Social conditions',20:'Management',21:'Case studies',
           22:'Education',23:'Economic policy',24:'Law and legislation',25:'Social sciences',
           26:'Research',27:'Finance',28:'哲學',29:'文集'}

In [ ]:
test_df = pd.read_csv("/content/drive/My Drive/111_CALISE_CDA111004/data/merge_1w_test_dataset_for_user.csv")
test_df

,Permanent Call Number,MMS Id,Publication Date,Resource Type,Title,Edition,Author,Author (contributor),ISBN,Place of Publication - Country,Publisher,Language Code,Material Type,Subjects
0,813.4 J27 P855,991015210069705721,1991.,Book - Physical,"The trial of curiosity : Henry James, William ...",NaN,"Posnock, Ross.",NaN,9780195066067; 0195066065; 9780195071245; 0195...,United States,Oxford University Press,eng,Book,United States
1,322.5098 B561,991009685179705721,1986.,Book - Physical,Sentinels of empire : the United States and La...,NaN,"Black, Jan Knippers, 1940-",NaN,031325155X; 9780313251559,United States,Greenwood Press,eng,Book,United States
2,327.73 C598,991014971839705721,c1995.,Book - Physical,After the crusade : American foreign policy fo...,NaN,"Clarke, Jonathan, 1947-","Clad, James.",1568330510; 9781568330518,United States,Madison Books,eng,Book,United States
3,371.2 G347,991014898179705721,c1981.,Book - Physical,Forces on leadership /,NaN,"Giammatteo, Michael C.","Giammatteo, Dolores M.",0882101161; 9780882101163,United States,National Association of Secondary School Princ...,eng,Book,United States
4,327.73047 S826,991015579919705721,2014.,Book - Physical,The limits of partnership : U.S.-Russian relat...,NaN,"Stent, Angela.",NaN,9780691152974; 0691152977,United States,Princeton University Press,eng,Book,United States; Foreign relations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917,520.932 8646,991004119739705721,2018[民107],Book - Physical,歷史課的攻防戰 : 成為全新歷史公民的韓國經驗 /,初版,金正仁 著,陳聖薇 譯; 邱麟翔 譯,9789578654099; 957865409X,China,遠足文化發行; 八旗文化出版,chi,Book,文集
9918,588.1607 8772,991012409189705721,2012[民101],Book - Physical,行政訴訟之研討 /,初版,NaN,行政訴訟研究會,9789868893108; 9868893100; 9789868893139; 9868...,China,台灣法學出版,chi,Book,文集
9919,921.07 057,991013140719705721,2003[民92],Book - Physical,中华百年祭.,第一版,NaN,田海林,7810418157; 9787810418157,China,河南大學,chi,Book,文集
9920,839.3207 013,991009637279705721,2011[民100],Book - Physical,戰鼓聲中的歌者 : 龍瑛宗及其同時代東亞作家論文集 /,第1版,下村作次郎,王惠珍,9789860280463; 9860280460,China,清大臺灣文學硏究所,chi,Book,文集


In [ ]:
#觀察原始長度
len(test_df)

9922

In [ ]:
#分離中英文書
chi_test_df= test_df[(test_df['Language Code']=='chi')]
eng_test_df= test_df[(test_df['Language Code']=='eng')]

##### 分類號

In [ ]:
#先處理分類號
#自然語言處理，分類號取前三碼
import re
def get_codenum(s):
  if type(s) is str:
    m = re.search("\d{3}",s)
    if m is not None:
      return m.group(0)
    else:
      return "x"
  else:
    return "not_str"

In [ ]:
#把分類號丟進去定義的自然語言處理的函數
eng_test_df['cut_eng_num']=eng_test_df['Permanent Call Number'].apply(get_codenum)
eng_test_df

,Permanent Call Number,MMS Id,Publication Date,Resource Type,Title,Edition,Author,Author (contributor),ISBN,Place of Publication - Country,Publisher,Language Code,Material Type,Subjects,cut_eng_num
0,813.4 J27 P855,991015210069705721,1991.,Book - Physical,"The trial of curiosity : Henry James, William ...",NaN,"Posnock, Ross.",NaN,9780195066067; 0195066065; 9780195071245; 0195...,United States,Oxford University Press,eng,Book,United States,813
1,322.5098 B561,991009685179705721,1986.,Book - Physical,Sentinels of empire : the United States and La...,NaN,"Black, Jan Knippers, 1940-",NaN,031325155X; 9780313251559,United States,Greenwood Press,eng,Book,United States,322
2,327.73 C598,991014971839705721,c1995.,Book - Physical,After the crusade : American foreign policy fo...,NaN,"Clarke, Jonathan, 1947-","Clad, James.",1568330510; 9781568330518,United States,Madison Books,eng,Book,United States,327
3,371.2 G347,991014898179705721,c1981.,Book - Physical,Forces on leadership /,NaN,"Giammatteo, Michael C.","Giammatteo, Dolores M.",0882101161; 9780882101163,United States,National Association of Secondary School Princ...,eng,Book,United States,371
4,327.73047 S826,991015579919705721,2014.,Book - Physical,The limits of partnership : U.S.-Russian relat...,NaN,"Stent, Angela.",NaN,9780691152974; 0691152977,United States,Princeton University Press,eng,Book,United States; Foreign relations,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9639,332.41 In8,991015830599705721,1983.,Book - Physical,The international transmission of inflation /,NaN,"Darby, Michael R.","Lothian, James R.",0226136418; 9780226136417,United States,University of Chicago Press,eng,Book,Finance,332
9640,332.6452 O62,991000298079705721,c1983.,Book - Physical,Option pricing : theory and applications /,NaN,NaN,Salomon Brothers Center for the Study of Finan...,0669057142; 9780669057140,United States,Lexington Books,eng,Book,Finance,332
9641,332.10952 A531,991015486289705721,c2004.,Book - Physical,Japan's financial crisis : institutional rigid...,NaN,"Amyx, Jennifer Ann.",NaN,9780691114477; 0691114471; 9780691128689; 0691...,United States,Princeton University Press,eng,Book,Finance,332
9642,346.73078 B218-III,991005602049705721,c2010.,Book - Physical,Bankruptcy and financial restructuring law 201...,NaN,NaN,NaN,9780314263483; 0314263489,United States,Aspatore,eng,Book,Finance,346


In [ ]:
#看看那些例外
eng_test_df[eng_test_df['cut_eng_num']=="x"]

,Permanent Call Number,MMS Id,Publication Date,Resource Type,Title,Edition,Author,Author (contributor),ISBN,Place of Publication - Country,Publisher,Language Code,Material Type,Subjects,cut_eng_num
65,Unknown,991021191859505721,©1987.,Book - Physical,Spike Lee's gotta have it : inside guerrilla f...,NaN,"Lee, Spike.",NaN,9780671644178; 0671644173,United States,Simon & Schuster,eng,Book,United States,x
77,Unknown,991021191848205721,1981.,Book - Physical,In struggle : SNCC and the Black awakening of ...,NaN,"Carson, Clayborne, 1944-",NaN,9780674447257; 0674447255; 9780674447264; 0674...,United States,Harvard University Press,eng,Book,United States,x
82,Unknown,991021186721505721,©1988.,Book - Physical,The radical renewal : the politics of ideas in...,1st ed.,"Birnbaum, Norman.",NaN,9780394523156; 0394523156; 9780394706597; 0394...,United States,Pantheon Books,eng,Book,United States,x
466,Unknown,991021215484205721,[2021],Book - Physical,"Useful objects : museums, science, and literat...",NaN,"Gochberg, Reed, author.",NaN,0197553486; 9780197553480; 9780197553510; 0197...,United States,Oxford University Press,eng,Book,United States,x
869,SE 1.6:P 94/2,991016346489705721,1992.,Book - Physical,Protecting investors : a half century of inves...,NaN,United States. Securities and Exchange Commiss...,NaN,0160379008; 9780160379000,United States,For sale by the US GPO Supt of Docs; Division ...,eng,Book,United States,x
982,Unknown,991021169855805721,[2018],Book - Physical,"""I'm not gonna die in this damn place"" : manli...",NaN,"Coronado, Juan David, author.",NaN,1611862728; 9781611862720; 9781609175542; 1609...,United States,Michigan State University Press,eng,Book,United States,x
1387,Unknown,991021220772805721,[2019],Book - Physical,Unfair labor? : American Indians and the 1893 ...,NaN,"Beck, David, 1956- author.",NaN,1496206835; 9781496206831; 1496214846; 9781496...,United States,University of Nebraska Press,eng,Book,History,x
1894,Unknown,991005144729705721,c1992.,Book - Physical,The lebanese in the world : a century of emigr...,NaN,NaN,ShehadiNadim.; HouraniAlbert.,1850433038; 9781850433033,United Kingdom,Centre for Lebanese Studies in association wit...,eng,Book,History,x
1926,15.75,991021169978305721,©1996.,Book - Physical,"Crime, society, and the state in the nineteent...",NaN,"Bankoff, Greg.",NaN,9789715502030; 9715502032,Philippines,Ateneo de Manila University Press,eng,Book,History,x
1964,Unknown,991021216972405721,[2022],Book - Physical,Pure and true : the everyday politics of ethni...,NaN,"Stroup, David R., author.",NaN,0295749822; 9780295749822; 0295749830; 9780295...,United States,University of Washington Press,eng,Book,History,x


In [ ]:
#單獨取出切完的分類號
eng_num=eng_test_df['cut_eng_num']

In [ ]:
#一樣的事情換中文分類號做
chi_test_df['cut_chi_num']=chi_test_df['Permanent Call Number'].apply(get_codenum)
chi_test_df

,Permanent Call Number,MMS Id,Publication Date,Resource Type,Title,Edition,Author,Author (contributor),ISBN,Place of Publication - Country,Publisher,Language Code,Material Type,Subjects,cut_chi_num
106,785.2885 8534 865,991016369389705721,民84.,Book - Physical,海上悍將海爾賽 /,初版,"波特 著 (Potter, E. B.)","Potter, E. B.; 伊斯曼 譯",9577083021; 9789577083029,China,麥田,chi,Book,United States,785
154,191.9 844:9,991009922079705721,1999[民88],Book - Physical,自然心藥 /,初版,"雷門 著 (Remen, Rachel Naomi)","Remen, Rachel Naomi.; 李成嶽 譯",9578269080; 9789578269088,China,學英文化總經銷; 成陽出版,chi,Book,United States,191
182,490.9952 854 8853,991015998019705721,民84.,Book - Physical,美國郵購傳奇 : 比恩的成功祕訣 /,初版,"蒙哥馬利 作 (Montgomery,M. R.)","Montgomery, M. R.; 陳建生 譯",9576792827; 9789576792823,China,方智,chi,Book,United States,490
219,785.289 8644,991009245339705721,1998[民87],Book - Physical,個人歷史 : 全美最有影響力的女報人葛蘭姆 /,第一版,"葛理翰 著 (Graham, Katharine)","Graham, Katharine.; 尹萍 譯",9576215072; 9789576215070; 9576215080; 9789576...,China,黎銘總經銷; 天下遠見出版,chi,Book,United States,785
237,521.7 8683,991015781619705721,2001[民90],Book - Physical,校長的課程領導 /,初版,"葛萊松 原著 (Glatthorn, Allan A.)","Glatthorn, Allan A.; 單文經 譯",9573070065; 9789573070061,China,學富文化,chi,Book,United States,521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917,520.932 8646,991004119739705721,2018[民107],Book - Physical,歷史課的攻防戰 : 成為全新歷史公民的韓國經驗 /,初版,金正仁 著,陳聖薇 譯; 邱麟翔 譯,9789578654099; 957865409X,China,遠足文化發行; 八旗文化出版,chi,Book,文集,520
9918,588.1607 8772,991012409189705721,2012[民101],Book - Physical,行政訴訟之研討 /,初版,NaN,行政訴訟研究會,9789868893108; 9868893100; 9789868893139; 9868...,China,台灣法學出版,chi,Book,文集,588
9919,921.07 057,991013140719705721,2003[民92],Book - Physical,中华百年祭.,第一版,NaN,田海林,7810418157; 9787810418157,China,河南大學,chi,Book,文集,921
9920,839.3207 013,991009637279705721,2011[民100],Book - Physical,戰鼓聲中的歌者 : 龍瑛宗及其同時代東亞作家論文集 /,第1版,下村作次郎,王惠珍,9789860280463; 9860280460,China,清大臺灣文學硏究所,chi,Book,文集,839


In [ ]:
#看看誰是x
chi_test_df[chi_test_df['cut_chi_num']=="x"]

,Permanent Call Number,MMS Id,Publication Date,Resource Type,Title,Edition,Author,Author (contributor),ISBN,Place of Publication - Country,Publisher,Language Code,Material Type,Subjects,cut_chi_num
1358,Unknown,991021198582505721,2020.,Book - Physical,Shu fang tian lu : hai wai Han ji shou cang sh...,Di 1 ban.,NaN,汉籍与汉学 : 海外汉籍收藏史研究学术研讨会 Beijing da xue) (2019 :...,7201160494; 9787201160498,China,Tianjin ren min chu ban she,chi,Book,United States,x
1381,Unknown,991021220638105721,2015,Book - Physical,<>.,NaN,NaN,NaN,983316448X; 9789833164486,Malaysia,NaN,chi,Book,History,x
1547,Unknown,991021226045605721,2021.,Book - Physical,"Qing dai Yunnan ""yi ren tu shuo"" yan jiu /",Beijing di 1 ban.,"Li, Li, 1970- author.","史青, 1977- author.; Shi, Qing, 1977- author.",7101153356; 9787101153354,China,Zhonghua shu ju,chi,Book,History,x
1866,Unknown,991021187721205721,2020.,Book - Physical,"Run se hong ye : ""Han shu"" wen ben de xing che...",Di 1 ban.,"Chen, Jun, author.",NaN,7301159773; 9787301159774,China,Beijing da xue chu ban she,chi,Book,History,x
3195,Unknown,991021212457405721,2022[民111],Book - Physical,"國學潮人誌. 10位最具才情的古潮人, 成敗起伏的生命中, 有哪些與眾不同的求生姿態、不同的...",初版,宋怡慧 著,NaN,9786263101647; 6263101644,China,家庭傳媒城邦分公司發行; 麥田出版,chi,Book,中國,x
3675,Unknown,991021187719805721,2021.,Book - Physical,Zhao ya : Qing dai xian ya de shu li yu cha yi /,Di 1 ban.,"Bai, Derui, author.","赖骏楠, translator.; Lai, Junnan, translator.; 尤陈...",7549576254; 9787549576258,China,Guangxi shi fan da xue chu ban she,chi,Book,Politics and government,x
4506,Unknown,991021214761405721,2021.,Book - Physical,Zhejiang da xue yi shu yu kao gu yan jiu. Li L...,Di 1 ban.,NaN,"浙江大学. 艺术与考古研究中心, editor.; Zhejiang da xue. Yi ...",7308209210; 9787308209212,China,Zhejiang da xue chu ban she,chi,Book,Congresses,x
5193,Unknown,991021216750705721,2021.,Book - Physical,Long qi piao yang de jian dui : Zhongguo jin d...,Di 1 ban.,"Jiang, Ming, 1957- author.",NaN,7559437583; 9787559437587,China,Jiangsu feng huang wen yi chu ban she,chi,Book,China,x
5351,Unknown,991021187715305721,2020.,Book - Physical,Zui hou de huang zu : Qing dai gong ting she h...,Di 1 ban.,"Rawski, Evelyn Sakakida, author.","周卫平, 1973- translator.; Zhou, Weiping, 1973- t...",7208159823; 9787208159822,China,Shanghai ren min chu ban she,chi,Book,China,x
5951,Unknown,991021143857805721,2017.,Book - Physical,Zhao Jiabi yu xian dai wen xue cong shu : Lian...,Di 1 ban.,"Zhang, Zexian, author.",NaN,7547612539; 9787547612538,China,Shanghai yuan dong chu ban she,chi,Book,History and criticism,x


In [ ]:
chi_num=chi_test_df['cut_chi_num']
chi_num

106     785
154     191
182     490
219     785
237     521
       ... 
9917    520
9918    588
9919    921
9920    839
9921    422
Name: cut_chi_num, Length: 2260, dtype: object

In [ ]:
#分類號encoding
from gensim.models import Word2Vec
from numpy import array
from keras.preprocessing.sequence import pad_sequences
#取出dataframe進入list
eng_num=list(eng_num)
chi_num=list(chi_num)

In [ ]:
#利用辭典取得該分類號有的主題詞 該分類號擁有的主題詞去加權平均，成為該分類號的向量
from gensim.models import KeyedVectors
wv = KeyedVectors.load('/content/drive/My Drive/2022大數據競賽/exploratory_data_analysis_py/主題詞CBOW.wv')
words = wv.vocab.keys()
cbow_class_dict = {word:wv[word] for word in words}

In [ ]:
#分離出主題詞
test_books_subjects = test_df['Subjects']
test_books_subjects.head(10)

0                       United States
1                       United States
2                       United States
3                       United States
4    United States; Foreign relations
5                       United States
6                       United States
7                       United States
8                       United States
9                       United States
Name: Subjects, dtype: object

In [ ]:
test_books_subjects1=list(test_books_subjects)
test_books_subjects2 = []
for i in range(len(test_books_subjects1)):
  single = test_books_subjects1[i].split('; ')
  test_books_subjects2.append(single)
test_books_subjects2

[['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States', 'Foreign relations'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States', 'Politics and government', 'Philosophy'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States']

In [ ]:
#分類號對應主題詞
d=zip (eng_num,test_books_subjects2)
temp=list(d)
temp

[('813', ['United States']),
 ('322', ['United States']),
 ('327', ['United States']),
 ('371', ['United States']),
 ('327', ['United States', 'Foreign relations']),
 ('328', ['United States']),
 ('362', ['United States']),
 ('344', ['United States']),
 ('343', ['United States']),
 ('370', ['United States']),
 ('912', ['United States']),
 ('333', ['United States']),
 ('365', ['United States']),
 ('303', ['United States']),
 ('327', ['United States']),
 ('327', ['United States']),
 ('909', ['United States']),
 ('342', ['United States']),
 ('342', ['United States']),
 ('973', ['United States']),
 ('327', ['United States']),
 ('344', ['United States']),
 ('332', ['United States']),
 ('331', ['United States']),
 ('353', ['United States']),
 ('303', ['United States', 'Politics and government', 'Philosophy']),
 ('324', ['United States']),
 ('331', ['United States']),
 ('331', ['United States']),
 ('342', ['United States']),
 ('327', ['United States']),
 ('658', ['United States']),
 ('327', [

In [ ]:
#加權平均分類號的空間向量
class_vec1=[]
for n in range(0,len(eng_num)):
  class_vec=[]
  for word in temp[n][1]:
    if word in cbow_class_dict.keys():
      class_vec.append(cbow_class_dict[word])
  arr1=np.array(class_vec)
  arr1=np.array(class_vec).mean(axis=0)
  class_vec1.append(arr1)
class_vec1

[array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16

In [ ]:
#加權平均分類號的空間向量
class_vec2=[]
for n in range(0,len(chi_num)):
  class_vec=[]
  for word in temp[n][1]:
    if word in cbow_class_dict.keys():
      class_vec.append(cbow_class_dict[word])
  arr1=np.array(class_vec)
  arr1=np.array(class_vec).mean(axis=0)
  class_vec2.append(arr1)
class_vec2

[array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16

In [ ]:
#合併中英文分類號embedding
class_vec=class_vec1+class_vec2
class_vec

[array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16582206, -0.8137172 ,
        -0.34042066,  0.43467477, -0.639687  ,  0.13514666, -0.7330261 ,
         0.16350318, -1.1438143 ,  0.7138114 ,  0.65704   ,  0.45791116,
         0.45791012,  0.9834203 , -0.4542936 , -0.5429781 , -0.6197546 ],
       dtype=float32),
 array([-0.84951997,  0.50277936,  0.6664511 ,  0.16

In [ ]:
#確認長度
len(class_vec)

9922

In [ ]:
#存檔
np.save('test_class_num.npy',class_vec)

##### 主題詞

In [ ]:
#分離出主題詞
test_books_subjects = test_df['Subjects']
test_books_subjects.head(10)

0                       United States
1                       United States
2                       United States
3                       United States
4    United States; Foreign relations
5                       United States
6                       United States
7                       United States
8                       United States
9                       United States
Name: Subjects, dtype: object

In [ ]:
books_subjects1=list(test_books_subjects)
books_subjects2 = []
for i in range(len(books_subjects1)):
  single = books_subjects1[i].split('; ')
  books_subjects2.append(single)
books_subjects2

[['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States', 'Foreign relations'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States', 'Politics and government', 'Philosophy'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States'],
 ['United States']

In [ ]:
# 載入word2vec物件與KeyVectors物件
from gensim.models import word2vec, KeyedVectors
# Settings
seed = 7777  # Seed for the random number generator.
sg = 0    # 1 for skip-gram; otherwise CBOW.
window_size = 99 # Maximum distance between the current and predicted word within a sentence (context window)
vector_size = 20 # Dimensionality of the word vectors
min_count = 5 # Ignores all words with total frequency lower than this
workers = 4 # Use these many worker threads to train the model (=faster training with multicore machines)
epochs = 10 # Number of iterations (epochs) over the corpus
batch_words = 10000 # Target size (in words) for batches of examples passed to worker threads

model_cbow = word2vec.Word2Vec(
    books_subjects2,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

cbow_word_vectors = model_cbow.wv # This object essentially contains the mapping between words and embeddings.

In [ ]:
#試看一下
cbow_word_vectors.most_similar("Economic policy", topn=30)

[('China', 0.4078379273414612),
 ('Philosophy', 0.37036511301994324),
 ('中國', 0.36864733695983887),
 ('歷史', 0.2866891026496887),
 ('Social conditions', 0.2204890102148056),
 ('Social aspects', 0.21184924244880676),
 ('Politics and government', 0.21029044687747955),
 ('論文', 0.1854739785194397),
 ('United States', 0.16526007652282715),
 ('文集', 0.15306468307971954),
 ('Psychology', 0.14859876036643982),
 ('Management', 0.09854915738105774),
 ('Social sciences', 0.09243165701627731),
 ('Economic conditions', 0.07814271748065948),
 ('Europe', 0.0025824233889579773),
 ('Biography', -0.025787293910980225),
 ('Education', -0.035395026206970215),
 ('臺灣', -0.04862812161445618),
 ('Finance', -0.13168905675411224),
 ('Foreign relations', -0.1916622519493103),
 ('History and criticism', -0.2025894820690155),
 ('Research', -0.20780318975448608),
 ('Law and legislation', -0.20977406203746796),
 ('History', -0.22392502427101135),
 ('哲學', -0.2579706609249115),
 ('Great Britain', -0.2940627634525299),
 

In [ ]:
cbow_word_vectors.wv.save(f'test_subject_CBOW.wv')

In [ ]:
#字典
words = model_cbow.wv.vocab.keys()
cbow_dict_s = {word:model_cbow.wv[word] for word in words}
cbow_dict_s

{'United States': array([-8.1310580e-03, -1.8188180e-02, -3.9946567e-04,  8.1156716e-03,
        -8.8797100e-03, -1.4336711e-02, -1.7083552e-02,  1.7229643e-02,
         1.2481364e-02, -1.5789786e-02, -1.1143533e-02, -6.8913255e-04,
        -4.6019300e-05,  5.6030084e-03, -9.4374232e-03,  6.6800681e-03,
        -1.6342608e-02, -2.6310806e-03,  2.0484969e-02, -1.2842078e-03],
       dtype=float32),
 'Foreign relations': array([ 0.02286594, -0.01944689,  0.00871662, -0.01042847,  0.01981004,
         0.00616256, -0.02109971,  0.02345687, -0.00697047, -0.00230449,
         0.01609075, -0.00240351,  0.00180943,  0.01028957,  0.00644025,
        -0.00544839, -0.0061489 ,  0.0163076 ,  0.01945364,  0.01823191],
       dtype=float32),
 'Politics and government': array([ 0.01496096, -0.00284879,  0.01830979,  0.02058277, -0.00046666,
        -0.0114152 , -0.00856107,  0.01488106,  0.01812525,  0.01089957,
        -0.01090248, -0.02051216,  0.01082942, -0.02505348,  0.00710652,
        -0.00327

In [ ]:
#存成npy
np.save('test_subjects.npy',cbow_dict_s)

In [ ]:
#subdict之前已定義好，檢視有沒有該書主題詞
ytrainlist = []
for i in range(len(books_subjects2)):
  temlist = []
  for j in range(len(subdict)):
    if subdict[j] in books_subjects2[i]:
      temlist.append(1)
    else:
      temlist.append(0)
  ytrainlist.append(temlist)
ytrainlist

[[1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [ ]:
#把它存進np array
y_train_numpy_array = np.array(ytrainlist)
np.save('test_y_train_numpy_array.npy',y_train_numpy_array )

##### 語言

In [ ]:
#分離出語言
language_code = test_df['Language Code']
language_code.head()

0    eng
1    eng
2    eng
3    eng
4    eng
Name: Language Code, dtype: object

In [ ]:
language_code1=list(language_code)
language_code2 = []
for i in range(len(language_code1)):
  if language_code1[i] == 'chi':
    language_code2.append(0)
  else:
    language_code2.append(1)
language_code2

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
np.save('test_language_code.npy',language_code2)

##### 書名

In [ ]:
#分離出書名
chi_titles=chi_test_df['Title']
eng_titles=eng_test_df['Title']

In [ ]:
#英文斷詞
#tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

#取出dataframe進入list
eng_titles1=list(eng_titles)

from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from gensim.parsing.preprocessing import remove_stopwords

#tokenization
i=0
eng_tokens3=[]
while i<len(eng_titles1):
  def listtostring(s):
      return ''.join(s)
  a=remove_stopwords(listtostring(eng_titles1[i]))
  a=listtostring(eng_titles1[i])
  a= re.sub(r'[^\w\s]', '', a)
  a=a.strip('/ ')
  a=a.strip('the')
  a=a.strip('The')
  a=nltk.word_tokenize(a)
  eng_tokens3.append(a)
  i+=1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()

def lemmatize(word):
    lemma = wnl.lemmatize(word)
    return lemma

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
eng_lem=[]

for i in range(0,len(eng_tokens3)):
  end=len(eng_tokens3[i])
  book_lem=[]
  for k in range(0,end):
    b=lemmatize(eng_tokens3[i][k])
    b=b.casefold()
    book_lem.append(b)
  eng_lem.append(book_lem)
eng_lem

[['trial',
  'of',
  'curiosity',
  'henry',
  'james',
  'william',
  'james',
  'and',
  'the',
  'challenge',
  'of',
  'modernity'],
 ['sentinels',
  'of',
  'empire',
  'the',
  'united',
  'states',
  'and',
  'latin',
  'american',
  'militarism'],
 ['after',
  'the',
  'crusade',
  'american',
  'foreign',
  'policy',
  'for',
  'the',
  'postsuperpower',
  'ag'],
 ['forces', 'on', 'leadership'],
 ['limit',
  'of',
  'partnership',
  'usrussian',
  'relation',
  'in',
  'the',
  'twentyfirst',
  'century'],
 ['divided',
  'we',
  'govern',
  'party',
  'control',
  'lawmaking',
  'and',
  'investigation',
  '19461990'],
 ['introduction', 'to', 'health', 'service'],
 ['federal',
  'law',
  'of',
  'employment',
  'discrimination',
  'in',
  'a',
  'nutshell'],
 ['real', 'estate', 'transaction', 'tax', 'planning', 'and', 'consequence'],
 ['effective', 'educational', 'environment'],
 ['west', 'point', 'atlas', 'of', 'american', 'war'],
 ['what',
  'a',
  'book',
  'can',
  'do',
 

In [ ]:
#lem優化!
eng_lem1 = []
for i in range(len(eng_lem)):
  llist = nltk.pos_tag(eng_lem[i])
  eng_lem1.append(llist)

In [ ]:
from nltk.corpus import wordnet as wn

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('JJ'):
        return wn.ADJ
    elif nltk_tag.startswith('VV'):
        return wn.VERB
    elif nltk_tag.startswith('NN'):
        return wn.NOUN
    elif nltk_tag.startswith('RR'):
        return wn.ADV
    else:
        return None

In [ ]:
#英文title
e_title = []
for i in range(len(eng_lem1)):
  lllist = []
  for j in range(len(eng_lem1[i])):
    if nltk_tag_to_wordnet_tag(eng_lem1[i][j][1]) is not None:
      lllist.append(wnl.lemmatize(eng_lem1[i][j][0],pos=nltk_tag_to_wordnet_tag(eng_lem1[i][j][1])))
    else:
      lllist.append(wnl.lemmatize(eng_lem1[i][j][0]))
  e_title.append(lllist)
e_title

[['trial',
  'of',
  'curiosity',
  'henry',
  'james',
  'william',
  'james',
  'and',
  'the',
  'challenge',
  'of',
  'modernity'],
 ['sentinel',
  'of',
  'empire',
  'the',
  'united',
  'state',
  'and',
  'latin',
  'american',
  'militarism'],
 ['after',
  'the',
  'crusade',
  'american',
  'foreign',
  'policy',
  'for',
  'the',
  'postsuperpower',
  'ag'],
 ['force', 'on', 'leadership'],
 ['limit',
  'of',
  'partnership',
  'usrussian',
  'relation',
  'in',
  'the',
  'twentyfirst',
  'century'],
 ['divided',
  'we',
  'govern',
  'party',
  'control',
  'lawmaking',
  'and',
  'investigation',
  '19461990'],
 ['introduction', 'to', 'health', 'service'],
 ['federal',
  'law',
  'of',
  'employment',
  'discrimination',
  'in',
  'a',
  'nutshell'],
 ['real', 'estate', 'transaction', 'tax', 'planning', 'and', 'consequence'],
 ['effective', 'educational', 'environment'],
 ['west', 'point', 'atlas', 'of', 'american', 'war'],
 ['what',
  'a',
  'book',
  'can',
  'do',
  't

In [ ]:
#中文斷詞
chi_titles=chi_test_df['Title']

In [ ]:
#取出dataframe進入list
chi_titles1=list(chi_titles)

In [ ]:
# 安裝中研院資訊所詞知識庫小組(ckip)的斷詞系統
!pip install -U ckiptagger[tf,gdown]

# 清除執行結果
display.clear_output()

In [ ]:
# 載入ckiptagger套件中所需程式
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
# 查看斷詞模型壓縮檔資料是否下載並解壓縮成功
os.listdir('/content/drive/My Drive/111_CALISE_CDA111004/ckiptagger_model/data')

['LICENSE',
 'embedding_character',
 'model_ws',
 'model_pos',
 'model_ner',
 'embedding_word']

In [ ]:
# 設定斷詞(ws)系統所使用的模型
ws = WS("/content/drive/My Drive/111_CALISE_CDA111004/ckiptagger_model/data")

In [ ]:
# 設定資料目錄
data_path = "/content/drive/My Drive/111_CALISE_CDA111004/data/書名斷詞"

In [ ]:
#開始斷詞
#rname = f'{data_path}/chi_titles01' # 讀取的檔案資料夾名稱
#print(f'將讀取{os.path.abspath(rname)}檔案資料夾')

wname = f'{data_path}/t_chi_titles_seg' # 寫入的檔案資料夾名稱
print(f'將寫入{os.path.abspath(wname)}檔案資料夾')

if os.path.exists(wname):
  print(f'{os.path.abspath(wname)}檔案資料夾已經存在')
else:
  print(f'{os.path.abspath(wname)}檔案資料夾不存在，將創建新資料夾')
  os.mkdir(wname)

將寫入/content/drive/My Drive/111_CALISE_CDA111004/data/書名斷詞/t_chi_titles_seg檔案資料夾
/content/drive/My Drive/111_CALISE_CDA111004/data/書名斷詞/t_chi_titles_seg檔案資料夾已經存在


In [ ]:
#去掉標點符號和空白
i=0
chi_titles2=[]
while i<len(chi_titles1):
  def listtostring(s):
      return ''.join(s)
  a=listtostring(chi_titles1[i])
  a= re.sub(r'[^\w\s]','', a)
  a=a.strip('/ ')
  chi_titles2.append(a)
  i+=1

In [ ]:
# 斷詞
words_list = ws(chi_titles2)
#清理斷詞
chi_tokens = []
m=0
n=0
for m in range(0,len(words_list)):
  end3=len(words_list[m])
  chi_tokens1=[]
  for n in range(0,end3):
      a=remove_stopwords(words_list[m][n].strip())
      a=words_list[m][n].strip()
      chi_tokens1.append(a)
      if len(a)==0:
        chi_tokens1.remove(a)
      a= ''.join(a).split()

  chi_tokens.append(chi_tokens1)
chi_tokens

[['海', '上', '悍將', '海爾賽'],
 ['自然心藥'],
 ['美國', '郵購', '傳奇', '比恩', '的', '成功', '祕訣'],
 ['個人', '歷史', '全', '美', '最', '有', '影響力', '的', '女', '報人', '葛蘭姆'],
 ['校長', '的', '課程', '領導'],
 ['回歸', '心靈', '本質', '創造', '21世紀', '新', '世界'],
 ['夢想', '無價'],
 ['彼得杜拉克', '的', '管理', '聖經'],
 ['期貨場', '內', '交易', '花招'],
 ['假日', '飯店', '傳奇', '成功', '一半', '運氣', '丶', '一半', '智慧'],
 ['大勢', '所', '趨', '改變', '我們', '生活', '的', '十', '個', '新', '方向'],
 ['知識', '與', '國富論'],
 ['經營',
  '之',
  '神',
  '求生',
  '寶典',
  '31',
  '位',
  '管理',
  '教父',
  '帶',
  '你',
  '渡過',
  '蕭條',
  '年代'],
 ['濃縮', '課程', '調整', '能力', '優異', '學生', '一般', '課程', '的', '全盤', '指引'],
 ['哈佛', '學者'],
 ['美國', '的', '軍事', '革新', '從', '越戰', '到', '波灣', '戰爭'],
 ['體育', '運動', '行政', '與', '管理'],
 ['權力', '轉移', '廿一世紀', '的', '戰略'],
 ['行動', '的', '勇氣', '危機', '與', '挑戰', '的', '回憶錄'],
 ['我', '的', '大英', '百科', '狂想曲'],
 ['問題', '解決', '的', '教', '與', '學'],
 ['社會', '工作', '價值', '與', '倫理'],
 ['兒童', '的', '一百', '種', '語文', '瑞吉歐艾蜜莉亞', '教育', '取向', '進一步', '的', '迴響'],
 ['教育', '行政', '研究', '手冊'],
 ['全', '是', '

In [ ]:
len(chi_tokens)

2260

In [ ]:
#中英文書名list融合
final_list= [*e_title,*chi_tokens]

In [ ]:
# 載入word2vec物件與KeyVectors物件
from gensim.models import word2vec, KeyedVectors

In [ ]:
#cbow
# Settings
seed = 60  # Seed for the random number generator.
sg = 0    # 1 for skip-gram; otherwise CBOW.
window_size = 5 # Maximum distance between the current and predicted word within a sentence (context window)
vector_size = 20 # Dimensionality of the word vectors
min_count = 5 # Ignores all words with total frequency lower than this
workers = 4 # Use these many worker threads to train the model (=faster training with multicore machines)
epochs = 10 # Number of iterations (epochs) over the corpus
batch_words = 10000 # Target size (in words) for batches of examples passed to worker threads

model_cbow = word2vec.Word2Vec(
    final_list,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

cbow_word_vectors = model_cbow.wv  # This object essentially contains the mapping between words and embeddings.

In [ ]:
print(model_cbow)

Word2Vec(vocab=2111, size=20, alpha=0.025)


In [ ]:
#存檔
cbow_word_vectors.save(f'test_title_CBOW.wv')

In [ ]:
#cbow dictionary
words = model_cbow.wv.vocab.keys()
cbow_dict = {word:model_cbow.wv[word] for word in words}

In [ ]:
#vector average
title_vec1=[]
error=[0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
error1 = np.array(error)
for n in range(0,len(final_list)):
  title_vec=[]
  for word in final_list[n]:
    #if len(final_list[n])==0:
      #title_vec.append(error)
    if word in cbow_dict.keys():
      title_vec.append(cbow_dict[word])
    #else:
    #  title_vec.append(error)
  if len(title_vec)==0:
    title_vec = error
    #print(n)
  #arr1=np.array(title_vec)
    arr1=error1
  else:
    arr1=np.array(title_vec).mean(axis=0)
  title_vec1.append(arr1)
title_vec1

[array([ 1.10408384e-02,  7.34910786e-01, -4.19667453e-01, -6.56128943e-01,
        -6.09611213e-01,  3.76128018e-01, -2.97319293e-01, -1.02919079e-01,
         3.62030528e-02,  3.66895944e-01,  7.95702338e-01,  4.28701043e-01,
        -8.68574008e-02,  4.25384969e-01, -1.06392491e+00, -1.20823300e+00,
         5.30948520e-01, -4.05872375e-01, -2.19181180e-04, -1.19633034e-01],
       dtype=float32),
 array([-0.10692741,  0.894833  ,  0.21313116, -1.2579798 , -1.0270367 ,
         0.38684505, -0.48908067,  0.11629079, -0.14086501,  0.12164547,
         1.0260546 ,  0.7271469 , -0.23192331,  0.70678115, -1.9504701 ,
        -1.2130419 ,  1.3656244 , -0.4238003 ,  0.11086535, -0.2684424 ],
       dtype=float32),
 array([-0.17964113,  0.83432406, -0.36612666, -0.91216975, -0.72212446,
         0.3191929 , -0.59967846, -0.10683579,  0.04857682,  0.47546688,
         0.77720946,  0.48951817, -0.28099358,  0.3376386 , -1.3097712 ,
        -1.2553536 ,  0.8691484 , -0.52466875,  0.16159245, -

In [ ]:
#檢查是否被扣光的書有被填入0.5
title_vec1[89]

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [ ]:
np.shape(title_vec1)

(9922, 20)

In [ ]:
np.save('test_title.npy',title_vec1)

## 匯入模型測試

In [ ]:
# import model
#environment setup
import statsmodels as sm
import sklearn as skl
import scipy as sp
import matplotlib

#deep learning libraries
import tensorflow as tf
import keras
from tensorflow.keras.layers import Embedding, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate, Dense
from tensorflow.keras.layers import LSTM

from gensim.models import word2vec, KeyedVectors
#import theano
#padsequence

# data process
# load in the model
model = keras.models.load_model("our_model.h5")
# print result
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 600)               25200     
                                                                 
 dense_4 (Dense)             (None, 800)               480800    
                                                                 
 dense_5 (Dense)             (None, 30)                24030     
                                                                 
Total params: 530,030
Trainable params: 530,030
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#load test data
x_class=np.load('test_class_num.npy')
x_language=np.load('test_language_code.npy')
x_titles=np.load('test_title.npy')

x_language=x_language.reshape(-1,1)

x_test=np.hstack((x_titles,x_language,x_class))

y_test = np.load('test_y_train_numpy_array.npy')

In [ ]:
#evaluate
eva = model.evaluate(x_test, y_test)
eva

311/311 [==============================] - 2s 5ms/step - loss: 0.0515 - accuracy: 0.7571


[0.051547445356845856, 0.7571054100990295]

In [ ]:
#預測
predict = model.predict(x_test)
predict

array([[9.8447913e-01, 2.5543832e-07, 6.4973783e-11, ..., 3.2876895e-04,
        8.2489368e-07, 2.1532249e-07],
       [9.9897921e-01, 5.9283270e-08, 1.4757194e-12, ..., 2.5775584e-05,
        5.4035407e-08, 8.2052702e-09],
       [9.9412704e-01, 1.4525124e-07, 1.8436325e-11, ..., 1.4132896e-04,
        2.6230038e-07, 7.8808618e-08],
       ...,
       [4.1416389e-08, 2.6353014e-01, 2.4230983e-04, ..., 1.1142930e-03,
        5.2209711e-01, 2.0249413e-01],
       [3.3333503e-08, 9.4535723e-02, 1.6454267e-04, ..., 8.9178339e-04,
        7.2564709e-01, 1.7161313e-01],
       [3.6402948e-08, 2.4092399e-01, 1.7782746e-04, ..., 8.3510636e-04,
        5.7381272e-01, 1.7634483e-01]], dtype=float32)

In [ ]:
#threshold=0.19
y_pred=[]
for sample in  predict:
  llist=[]

  for i in sample:
    if i>=0.19:
      llist.append(1)
    else:
      llist.append(0)
  if llist == [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]:
    llist = []
    biggest = 0
    for j in sample:
      if j>biggest:
        biggest = j
    for k in sample:
      if k==biggest:
       llist.append(1)
      else:
       llist.append(0)
  llist = np.array(llist)
  y_pred.append(llist)
y_pred = np.array(y_pred)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0]])

In [ ]:
y_pred_word = []
for i in range(len(y_pred)):
  temp = []
  for j in range(len(y_pred[i])):
    if y_pred[i][j]==1:
      temp.append(subdict[j])
      temp1=np.array(temp)
  y_pred_word.append(temp1)
y_pred_word = np.array(y_pred_word)
y_pred_word

array([array(['United States'], dtype='<U13'),
       array(['United States'], dtype='<U13'),
       array(['United States'], dtype='<U13'), ...,
       array(['History', '哲學', '文集'], dtype='<U7'),
       array(['哲學'], dtype='<U2'), array(['History', '哲學'], dtype='<U7')],
      dtype=object)

# 2.評估指標(Evaluation)
請使用 F1-Score、Recall、Precision 針對您的模型進行評估，並印出結果。

In [ ]:
y_pred=np.array(y_pred)
y_test=np.array(y_test)

In [ ]:
def TP(y_test, y_pred):
  count=0
  temp=[]
  for i in range(0,len(y_pred)):
    for n in range(0,30):
      if y_pred[i][n]==1 and y_test[i][n]==1:
        count = count + 1
  return count

In [ ]:
def TN(y_test, y_pred):
  count=0
  for i in range(0,len(y_pred)):
    for n in range(0,30):
      if y_pred[i][n]==0 and y_test[i][n]==0:
        count+=1
  return count

In [ ]:
def FP(y_test, y_pred):
  count=0
  for i in range(0,len(y_pred)):
    for n in range(0,30):
      if y_pred[i][n]==1 and y_test[i][n]==0:
        count+=1
  return count

In [ ]:
def FN(y_test, y_pred):
  count=0
  for i in range(0,len(y_pred)):
    for n in range(0,30):
      if y_pred[i][n]==0 and y_test[i][n]==1:
        count+=1
  return count

In [ ]:
tp=TP(y_test,y_pred)
print(tp)
tn=TN(y_test,y_pred)
print(tn)
fp=FP(y_test, y_pred)
print(fp)
fn=FN(y_test, y_pred)
print(fn)

7810
285015
2595
2240


In [ ]:
test_precision=tp/(tp+fp)
test_precision

0.7506006727534839

In [ ]:
test_recall=tp/(tp+fn)
test_recall

0.7771144278606965

In [ ]:
#F1
F1 = 2 * (test_precision * test_recall)/(test_precision + test_recall)
F1

0.7636274749450013